In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
import pandas as pd

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SHARED_STORE, SNOBAL_DIR, SNOTEL_DIR, DATA_DIR, COARSEN_OPTS, HV_PLOT_OPTS

In [ ]:
client = start_cluster(6, 24)
client_ip_and_port(client)

## SNOTEL 

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites.json')

## Model run with HRRR clouds and solar

In [ ]:
water_year = 'wy2021'

HRRR_solar = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/net_solar.nc',
    parallel=True, chunks={'time': 24, 'y' :10, 'x': 10},
    drop_variables=['DSWRF', 'illumination_angle', 'zenith', 'azimuth', 'albedo', 'albedo_vis', 'albedo_ir'],
)

butte_snobal_hrrr = HRRR_solar.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
schofield_snobal_hrrr = HRRR_solar.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
taylor_snobal_hrrr = HRRR_solar.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()
irwin_snobal_hrrr = HRRR_solar.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').squeeze(['x', 'y']).compute()

# water_year = 'wy2022'
# HRRR_solar = xr.open_mfdataset(
#     f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/net_solar.nc',
#     parallel=True, chunks={'time': 24, 'y' :10, 'x': 10},
#     drop_variables=['DSWRF', 'illumination_angle', 'zenith', 'azimuth', 'albedo', 'albedo_vis', 'albedo_ir'],
# )

# butte_snobal_hrrr = xr.merge([
#     butte_snobal_hrrr,
#     HRRR_solar.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
# ])
# schofield_snobal_hrrr = xr.merge([
#     schofield_snobal_hrrr,
#     HRRR_solar.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
# ])
# taylor_snobal_hrrr = xr.merge([
#     taylor_snobal_hrrr,    
#     HRRR_solar.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()
# ])
# irwin_snobal_hrrr = xr.merge([
#     irwin_snobal_hrrr,
#     HRRR_solar.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').squeeze(['x', 'y']).compute()
# ])

del HRRR_solar

## Model run with HRRR clouds and solar + MODIS albedo

In [ ]:
water_year = 'wy2021'

HRRR_solar = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar_modis/run*/net_solar.nc',
    parallel=True, chunks={'time': 24, 'y' :10, 'x': 10},
    drop_variables=['DSWRF', 'illumination_angle', 'albedo'],
)

butte_snobal_hrrr_modis = HRRR_solar.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
schofield_snobal_hrrr_modis = HRRR_solar.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
taylor_snobal_hrrr_modis = HRRR_solar.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()
irwin_snobal_hrrr_modis = HRRR_solar.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').squeeze(['x', 'y']).compute()

# water_year = 'wy2022'
# HRRR_solar = xr.open_mfdataset(
#     f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/net_solar.nc',
#     parallel=True, chunks={'time': 24, 'y' :10, 'x': 10},
#     drop_variables=['DSWRF', 'illumination_angle', 'zenith', 'azimuth', 'albedo', 'albedo_vis', 'albedo_ir'],
# )

# butte_snobal_hrrr = xr.merge([
#     butte_snobal_hrrr,
#     HRRR_solar.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
# ])
# schofield_snobal_hrrr = xr.merge([
#     schofield_snobal_hrrr,
#     HRRR_solar.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
# ])
# taylor_snobal_hrrr = xr.merge([
#     taylor_snobal_hrrr,    
#     HRRR_solar.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()
# ])
# irwin_snobal_hrrr = xr.merge([
#     irwin_snobal_hrrr,
#     HRRR_solar.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').squeeze(['x', 'y']).compute()
# ])

del HRRR_solar

In [ ]:
water_year = 'wy2021'

## Model with SMRF clouds and solar 

In [ ]:
smrf_eb = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw/run*/smrf_energy*.nc',
    parallel=True, chunks={'time': 24, 'y' :10, 'x': 10},
    drop_variables=['albedo_ir', 'albedo_vis'],
)

# Net Solar

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Net Solar W/m^2'

In [ ]:
butte_snobal_hrrr = butte_snobal_hrrr.net_solar
schofield_snobal_hrrr = schofield_snobal_hrrr.net_solar
taylor_snobal_hrrr = taylor_snobal_hrrr.net_solar
irwin_snobal_hrrr = irwin_snobal_hrrr.net_solar

In [ ]:
butte_snobal_hrrr_modis = butte_snobal_hrrr_modis.net_solar
schofield_snobal_hrrr_modis = schofield_snobal_hrrr_modis.net_solar
taylor_snobal_hrrr_modis = taylor_snobal_hrrr_modis.net_solar
irwin_snobal_hrrr_modis = irwin_snobal_hrrr_modis.net_solar

In [ ]:
butte_snobal = smrf_eb.net_solar.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
schofield_snobal = smrf_eb.net_solar.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
taylor_snobal = smrf_eb.net_solar.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()
irwin_snobal = smrf_eb.net_solar.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').squeeze(['x', 'y']).compute()

### Schofield 

In [ ]:
schofield_snobal_hrrr.hvplot(label='HRRR solar').opts(**HV_PLOT_OPTS) * \
schofield_snobal_hrrr_modis.hvplot(label='HRRR + MODIS') * \
schofield_snobal.hvplot(label='SMRF solar') 

### Butte

In [ ]:
butte_snobal_hrrr.hvplot(label='HRRR solar').opts(**HV_PLOT_OPTS) * \
butte_snobal_hrrr.hvplot(label='HRRR + MODIS') * \
butte_snobal.hvplot(label='SMRF solar') 

### Taylor 

In [ ]:
taylor_snobal_hrrr.hvplot(label='HRRR solar').opts(**HV_PLOT_OPTS) * \
taylor_snobal_hrrr_modis.hvplot(label='HRRR + MODIS') * \
taylor_snobal.hvplot(label='SMRF solar') 

### Irwin 

In [ ]:
import hvplot.pandas
pd.options.plotting.backend = 'holoviews'

In [ ]:
PD_OPTS = dict(
    parse_dates=True,
    index_col=0,
    names=['Temp', 'VIS_IN', 'NIR_IN', 'VIS_OUT', 'NIR_OUT'],
    header=0,
)

irwin_pyra = pd.concat([
    pd.read_csv(DATA_DIR / 'Snotel/wy2021/Irwin/2021-Irwin.csv', **PD_OPTS),
    # pd.read_csv(DATA_DIR / 'Snotel/wy2022/Irwin/2022-Irwin.csv', **PD_OPTS),
])

irwin_pyra = irwin_pyra.fillna({'VIS_IN': 0}).astype(np.float64)

# Filter unrealistic values
irwin_pyra[irwin_pyra < 0] = 0
irwin_pyra[irwin_pyra > 1300] = 1300
## no OUTCOMING when there is no INCOMING
irwin_pyra.loc[irwin_pyra['VIS_IN'] <= 0, 'VIS_OUT'] = 0
irwin_pyra.loc[irwin_pyra['NIR_IN'] <= 0, 'NIR_OUT'] = 0
## no OUTCOMING larger than INCOMING
irwin_pyra['VIS_OUT'] = irwin_pyra.apply(lambda r: r['VIS_OUT'] if r['VIS_OUT'] < r['VIS_IN'] else r['VIS_IN'], axis=1)
irwin_pyra['NIR_OUT'] = irwin_pyra.apply(lambda r: r['NIR_OUT'] if r['NIR_OUT'] < r['NIR_IN'] else r['NIR_IN'], axis=1)

In [ ]:
ROLL_WINDOW=96 # Data is measured every 15 mins

In [ ]:
irwin_pyra['net_solar'] = (
    irwin_pyra['VIS_IN'].rolling(ROLL_WINDOW).mean().interpolate('time') + 
    irwin_pyra['NIR_IN'].rolling(ROLL_WINDOW).mean().interpolate('time') - 
    irwin_pyra['VIS_OUT'].rolling(ROLL_WINDOW).mean().interpolate('time') - 
    irwin_pyra['NIR_OUT'].rolling(ROLL_WINDOW).mean().interpolate('time')
)

#### Station data 

(
    irwin_pyra['VIS_IN'].rolling(ROLL_WINDOW).mean().interpolate('time') - 
    irwin_pyra['VIS_OUT'].rolling(ROLL_WINDOW).mean().interpolate('time')
).hvplot(label='VIS 1', **HV_PLOT_OPTS) * \
(
    irwin_pyra['NIR_IN'].rolling(ROLL_WINDOW).mean().interpolate('time') - 
    irwin_pyra['NIR_OUT'].rolling(ROLL_WINDOW).mean().interpolate('time')
).hvplot(label='NIR', **HV_PLOT_OPTS)

#### HRRR vs SMRF 

In [ ]:
irwin_snobal_hrrr.hvplot(label='HRRR solar').opts(**HV_PLOT_OPTS) * \
irwin_snobal_hrrr_modis.hvplot(label='HRRR + MODIS') * \
irwin_snobal.hvplot(label='SMRF solar') 

### HRRR vs SMRF vs Irwin station 

Station data is a daily rolling window and interpolated over the time axis

In [ ]:
irwin_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR', **HV_PLOT_OPTS) * \
irwin_snobal_hrrr_modis.rolling(time=24, center=True).mean().hvplot(label='HRRR + MODIS') * \
irwin_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF') * \
irwin_pyra['net_solar'].hvplot(label='Irwin Guides')

### Rolling (daily 

In [ ]:
(schofield_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Butte') * \
taylor_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Taylor') * \
irwin_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Irwin') + \
schofield_snobal_hrrr_modis.rolling(time=24, center=True).mean().hvplot(label='HRRR + MODIS Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal_hrrr_modis.rolling(time=24, center=True).mean().hvplot(label='HRRR + MODIS Butte') * \
taylor_snobal_hrrr_modis.rolling(time=24, center=True).mean().hvplot(label='HRRR + MODIS Taylor') * \
irwin_snobal_hrrr_modis.rolling(time=24, center=True).mean().hvplot(label='HRRR + MODIS Irwin') + \
schofield_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Schofield').opts(**HV_PLOT_OPTS) * \
butte_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Butte')  * \
taylor_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Taylor') * \
irwin_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Irwin')).cols(1)